In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import sqlite3
import os
import matplotlib.pyplot as plt
from sklearn import preprocessing

In [ ]:
print(os.getcwd())
os.chdir('./../')
print(os.getcwd())

# Escalado en estaciones
Aplicación de transformaciones de variables para analizar los datos de estaciones.

El escalado tiene como objetivo obtener generalizaciones mejores en el modelado.

In [ ]:
dfe = pd.read_csv('./data/interim/estaciones.csv')
dfe.tail()

[Preprocesamiento con Scikitlearn, escalado lineal, no lineal, tranformaciones, codificación de categorías](https://qu4nt.github.io/sklearn-doc-es/modules/preprocessing.html#preprocessing-data)

La normalización lineal uniforme: $v' = \frac{v - \min}{\max - \min}$ es un ejemplo en el que los datos pasan a estar entre 0 y 1.

In [ ]:
atributo = 'uso_bici'
# Ejemplo de normalización
dfe[f"{atributo}_norm"] = (dfe[atributo] - dfe[atributo].min()) / (dfe[atributo].max() - dfe[atributo].min())
# Ejemplo transformaciones
dfe[f"{atributo}_normed"] = (dfe[atributo] - dfe[atributo].mean())
dfe[f"{atributo}_tip"] = (dfe[atributo] - dfe[atributo].mean()) / dfe[atributo].std()
dfe[f"{atributo}_sqrt"] = np.sqrt(dfe[atributo])
dfe[f"{atributo}_pow"] = np.pow(dfe[atributo], 2)

In [ ]:
fig, ax = plt.subplots(3, 2, figsize=(6, 5))
axs = ax.flatten()
ax = dfe[atributo].hist(bins=20, ax=axs[0])
ax.set_title('Original')
ax = dfe[f"{atributo}_norm"].hist(bins=20, ax=axs[1], color='g')
ax.set_title('Normalizado')
ax = dfe[f"{atributo}_normed"].hist(bins=20, ax=axs[2], color='gray')
ax.set_title('Normalizado media cero')
ax = dfe[f"{atributo}_tip"].hist(bins=20, ax=axs[3], color='y')
ax.set_title('Tipificar')
ax = dfe[f"{atributo}_sqrt"].hist(bins=20, ax=axs[4], color='orange')
ax.set_title('Raíz')
ax = dfe[f"{atributo}_pow"].hist(bins=20, ax=axs[5], color='pink')
ax.set_title('Potencia')
fig.tight_layout()
plt.show()

In [ ]:
# Revisamos las unidades
x_feat, y_feat, size = 'lon', 'lat', 'uso_bici'
fig, ax = plt.subplots(1, 1, figsize=(5, 4))
ax = ax.scatter(dfe[x_feat], dfe[y_feat], 
                s=(dfe[size]*0.05), c=dfe[size], cmap='viridis')
plt.ylabel(y_feat)
plt.xlabel(x_feat)
fig.tight_layout()
cbar = plt.colorbar(ax)
cbar.set_label(size)
plt.show()

In [ ]:
x_feat, y_feat, size = 'lon', 'lat', 'uso_bici'
# Trabajamos con la librería scikit-learn
# prep_scaling = preprocessing.MinMaxScaler()
# prep_scaling = preprocessing.Normalizer("l1")
# prep_scaling = preprocessing.StandardScaler()
# prep_scaling = preprocessing.MaxAbsScaler()
# prep_scaling = preprocessing.RobustScaler()
# prep_scaling = preprocessing.QuantileTransformer(n_quantiles=6)

# Primero se entrena/ajusta el transformador
transf = prep_scaling.fit(dfe.loc[:, [x_feat, y_feat, size]].values)

# Luego se transforman los datos
data_transf = transf.transform(dfe.loc[:, [x_feat, y_feat, size]].values)

# Creamos el dataframe con el resultado
df2gm = pd.DataFrame(data_transf, 
                     columns=[x_feat, y_feat, size])
df2gm.head()

In [ ]:
x_feat, y_feat, size = 'lon', 'lat', 'uso_bici'
fig, ax = plt.subplots(1, 1, figsize=(5, 4))
ax = ax.scatter(df2gm[x_feat], df2gm[y_feat], 
                s=(df2gm[size]+1)**2, c=df2gm[size],
                cmap='viridis')
fig.suptitle("Transformación por cuantiles - 5")
plt.ylabel(y_feat)
plt.xlabel(x_feat)
fig.tight_layout()
cbar = plt.colorbar(ax)
cbar.set_label(size)
plt.show()

In [ ]:
x_feat, y_feat, size = 'lon', 'lat', 'uso_bici'
standard_scaler = preprocessing.StandardScaler()
min_max_scaler = preprocessing.MinMaxScaler()
# Definir el parámetro de quantiles
quantile_scaler = preprocessing.QuantileTransformer(n_quantiles=5)
normalizer = preprocessing.Normalizer()
dfe[[x_feat, y_feat, size]] = standard_scaler.fit_transform(dfe[[x_feat, y_feat, size]].values)
minmax_data = min_max_scaler.fit_transform(dfe[[x_feat, y_feat, size]].values)
quantile_data = quantile_scaler.fit_transform(dfe[[x_feat, y_feat, size]].values)
normalize_data = normalizer.fit_transform(dfe[[x_feat, y_feat, size]].values)

# Create subplots
fig, axs = plt.subplots(2, 2, figsize=(10, 10))
multip = 3
# Plot the data on each subplot
axs[0, 0].scatter(minmax_data[:,0], minmax_data[:,1],
                  label='MinMaxScaler', s=(dfe[size]+1)**multip, 
                  c=dfe[size], cmap='viridis')
axs[0, 0].set_title('MinMaxScaler')
axs[0, 0].legend()

axs[0, 1].scatter(quantile_data[:,0], quantile_data[:,1], 
                  label='QuantileTransformer', s=(dfe[size]+1)**multip, 
                  c=dfe[size], cmap='viridis')
axs[0, 1].set_title('QuantileTransformer')
axs[0, 1].legend()

axs[1, 0].scatter(normalize_data[:,0], normalize_data[:,1], 
                  label='Normalizer', s=(dfe[size]+1)**multip, 
                  c=dfe[size], cmap='viridis')
axs[1, 0].set_title('Normalizer')
axs[1, 0].legend()

# Original data in yellow is plotted on the last subplot
axs[1, 1].scatter(dfe['lon'], dfe['lat'], label='StandardScaler', 
                  s=(dfe[size]+1)**multip, c=dfe[size], cmap='viridis')  
axs[1, 1].set_title('StandardScaler')
axs[1, 1].legend()

plt.show()

In [ ]:
df.to_csv('./data/processed/usobarriosmeteo.csv', index=False)

# Uso de bicis
En el dataset objetivo "usobarriosmeteo.csv" vamos a transformar los datos para ampliar características.
> Hipótesis: la hora del día puede ser un factor que defina el comportamiento del cliente según la zona de la ciudad

In [ ]:
df = pd.read_csv('./data/interim/usobarriosmeteo.csv')
df.tail()

In [ ]:
# Transformar a datetime si no tiene el tipo 
df.dtypes

In [ ]:
df['fecha'] = pd.to_datetime(df['fecha'])
df['hora'] = pd.to_datetime(df['fecha']).dt.hour
df['dia'] = pd.to_datetime(df['fecha']).dt.day
df['mes'] = pd.to_datetime(df['fecha']).dt.month
df['anio'] = pd.to_datetime(df['fecha']).dt.year
df['dia_nombre'] = pd.to_datetime(df['fecha']).dt.day_name()
# Monday=0, Sunday=6
df['dia_semana'] = df['fecha'].dt.day_of_week
df['findesemana'] = np.where(df['dia_semana']>4, 1, 0)

In [ ]:
df.columns

In [ ]:
df['dia_nombre'].value_counts()

In [ ]:
df.to_csv('./data/processed/usobarriosmeteo.csv', index=False)

# Conclusiones
- Se tiene exploran las opciones para transformar datos
- El escalado puede ser útil para datos con unidades muy dispares
- La extracción de características puede mejorar el aprendizaje automático